In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json

import pandas as pd
from transformers import AutoTokenizer

from rate_severity_of_toxic_comments.utilities import *
from rate_severity_of_toxic_comments.model import *
from rate_severity_of_toxic_comments.dataset import *
from rate_severity_of_toxic_comments.training import * 
from rate_severity_of_toxic_comments.preprocessing import *
from rate_severity_of_toxic_comments.tokenizer import * 
from rate_severity_of_toxic_comments.embedding import * 

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
DEFAULT_CONFIG_FILE_PATH = "config/default.json"
LOCAL_CONFIG_FILE_PATH = "config/local.json"

default = open(DEFAULT_CONFIG_FILE_PATH)
CONFIG = json.load(default)

if os.path.exists(LOCAL_CONFIG_FILE_PATH):
    with open(LOCAL_CONFIG_FILE_PATH) as local:
        CONFIG.update(json.load(local))

CONFIG = process_config(CONFIG)

In [ ]:
df = pd.read_csv("res/data/weighted_train_small.csv")
# df = pd.read_csv("res/data/validation_data.csv")
df = df.sample(20)
data_size = len(df.index)

In [ ]:
train_split = 0.7
threshold_index = int(train_split * data_size)

df_train = df[:threshold_index].reset_index(drop=True)
df_valid = df[threshold_index:].reset_index(drop=True)

In [ ]:
preproc_pipelines = CONFIG['preprocessing']

### JUST FOR TESTING APPLIED ALSO ON VALIDATION SET

for df in [df_train, df_valid]:
    df = apply_preprocessing_pipelines(df, preproc_pipelines)

In [ ]:
training_data, val_data = build_datasets([df_train, df_valid], CONFIG, ["weighted", "weighted"])

In [ ]:
model, loss_history = run_training(training_data, val_data, log_interval=10, config=CONFIG, verbose=True)

In [ ]:
# TODO Evaluation and plotting